In [1]:
import os

import chainlit as cl
from dotenv import find_dotenv, load_dotenv
from langchain.callbacks import ContextCallbackHandler
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.memory import ConversationTokenBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from langchain_community.callbacks import ContextCallbackHandler
from langchain_community.vectorstores import Chroma
from langchain_core.documents.base import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from prompts import load_query_gen_prompt, load_raceng_prompt

load_dotenv(find_dotenv())
api_key = os.environ["OPENAI_API_KEY"]

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(load_query_gen_prompt())

In [2]:
# def init():
# define model and memory
llm = ChatOpenAI(api_key=api_key, verbose=True, streaming=True)
memory = ConversationTokenBufferMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True,
    input_key="question",
    max_token_limit=1000,
)
context_callback = ContextCallbackHandler()


# load documents, vectorize them into chroma, define doc retriever
files_to_read = [
    "datafiles/baseline_setup.txt",
    "datafiles/coachdave_tuning_guide.txt",
]
files_raw = []

for file in files_to_read:
    with open(file, "r", encoding="utf-8") as f:
        files_raw.append(f.readlines())

file_data = [(" ".join(f)) for f in files_raw]
docs = [Document(f,metadata={"source":m}) for f,m in zip(file_data,files_to_read)]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# some prompt engineering stuff goes here that i dont really understand
messages = [SystemMessagePromptTemplate.from_template(load_raceng_prompt())]
# print('mem', user_session.get('memory'))
messages.append(HumanMessagePromptTemplate.from_template("{question}"))
prompt = ChatPromptTemplate.from_messages(messages)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT, verbose=True)

doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff", verbose=True,prompt=prompt)


# define the retrieval chain
chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    verbose=True,
    memory=memory,
    rephrase_question=False,
    callbacks=[context_callback],
)

2024-03-08 21:25:36 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-03-08 21:25:36 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [3]:
vectorstore.get()["metadatas"]

[{'source': 'datafiles/baseline_setup.txt'},
 {'source': 'datafiles/baseline_setup.txt'},
 {'source': 'datafiles/baseline_setup.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'datafiles/coachdave_tuning_guide.txt'},
 {'source': 'data

In [4]:
chain.invoke({"question": "do you read me?"})



> Entering new ConversationalRetrievalChain chain...
2024-03-08 21:25:37 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a race engineer. You specialize in tuning vehicles for sim racing games.
    Question: do you read me?
    Source:
    ---------------------
        Content: Now that you have a full grasp on how to set up a car you are on your way to becoming a faster more consistent driver. However, many of the skills needed to truly understand a car’s setup will take time and experience. If you are still having issues The Coach Dave Academy has many other guides and resources to help you get the most out of your car and setup. One of the most useful guides is “How to fix oversteer and understeer in ACC” and “How to set up your car for the Wet in Assetto Corsa Competizione”.
Source: datafiles/coachdave_tuning_guide.txt

Content: Gre

{'question': 'do you read me?',
 'chat_history': [HumanMessage(content='do you read me?'),
  AIMessage(content='Yes, I read you. How can I assist you today?')],
 'answer': 'Yes, I read you. How can I assist you today?'}